# Import dependancies

In [1]:
import pandas as pd
from math import pi

In [2]:
!pip install plotly --upgrade --quiet
# Need latest version of plotly and need to pass renderers like this, for plotly to work in colab.
# Else you need to manually code a function, as plotly requires custom initialization.

In [3]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.graph_objects as go

# Insert the data for the observations of both O.C. Test and S.C. Test, then display the same

In [4]:
V_o = 115.8
I_o = 0.99
W_o = 20.06 # in Watts

V_sc = 9.5
I_sc = 8.87
W_sc = 66.64 # in Watts

df_oc_dict = {
  'S.No.': 1,
  'Vₒ (V)': V_o,
  'Iₒ (A)': I_o,
  'Wₒ = W x M.F (W)': W_o,
}

df_sc_dict = {
  'S.No.': 1,
  'V_sc (V)': V_sc,
  'I_sc (A)': I_sc,
  'W_sc = W x M.F (W)': W_sc,
}

index_values = [0]

df_oc = pd.DataFrame(df_oc_dict, index=index_values)
df_oc.set_index('S.No.', inplace=True)

df_sc = pd.DataFrame(df_sc_dict, index=index_values)
df_sc.set_index('S.No.', inplace=True)

display(df_oc)
print('\n')
display(df_sc)

,Vₒ (V),Iₒ (A),Wₒ = W x M.F (W)
S.No.,,,
1,115.8,0.99,20.06


,V_sc (V),I_sc (A),W_sc = W x M.F (W)
S.No.,,,
1,9.5,8.87,66.64


# Efficiency vs Load Characteristic at 0.8 pf lag

In [5]:
K = [0.15, 0.30, 0.45, 0.60, 0.75, 0.90, 1.05, 1.20]

rated_kva = 2
pf = 0.8 # lagging

n = len(K)
S_No = list(range(1, n + 1))

df_dict = {
  'S.No.': S_No,
  'Different Transformer Loading (K)': K,
}

df1 = pd.DataFrame(df_dict)
df1.set_index('S.No.', inplace=True)

df1['Copper Loss, K².P_sc (W)'] = df1['Different Transformer Loading (K)'] * W_sc
df1['Iron Loss, P_i (W)'] = W_o
df1['Total Loss, P_L = Iron Loss + Copper Loss (W)'] = df1['Copper Loss, K².P_sc (W)'] + df1['Iron Loss, P_i (W)']
df1['Output Power, P_o (kW)'] = df1['Different Transformer Loading (K)'] * rated_kva * pf
df1['Input Power, P_i (kW)'] = df1['Output Power, P_o (kW)'] + (df1['Total Loss, P_L = Iron Loss + Copper Loss (W)']/1000)
df1['Efficiency, % η'] = df1['Output Power, P_o (kW)'] / df1['Input Power, P_i (kW)'] * 100

display(df1.T)

S.No.,1,2,3,4,5,6,7,8
Different Transformer Loading (K),0.150000,0.300000,0.450000,0.600000,0.750000,0.900000,1.050000,1.200000
"Copper Loss, K².P_sc (W)",9.996000,19.992000,29.988000,39.984000,49.980000,59.976000,69.972000,79.968000
"Iron Loss, P_i (W)",20.060000,20.060000,20.060000,20.060000,20.060000,20.060000,20.060000,20.060000
"Total Loss, P_L = Iron Loss + Copper Loss (W)",30.056000,40.052000,50.048000,60.044000,70.040000,80.036000,90.032000,100.028000
"Output Power, P_o (kW)",0.240000,0.480000,0.720000,0.960000,1.200000,1.440000,1.680000,1.920000
"Input Power, P_i (kW)",0.270056,0.520052,0.770048,1.020044,1.270040,1.520036,1.770032,2.020028
"Efficiency, % η",88.870456,92.298462,93.500665,94.113587,94.485213,94.734598,94.913538,95.048187


# Regulation vs Power Factor

In [6]:
lv_rated_voltage = 115
hv_rated_voltage = 230
transformation_ratio = lv_rated_voltage / hv_rated_voltage

R_HV = W_sc / (I_sc ** 2)
Z_HV = V_sc / I_sc
X_HV = ((Z_HV ** 2) - (R_HV ** 2)) ** 0.5

X_LV = (transformation_ratio ** 2) * X_HV
R_LV = (transformation_ratio ** 2) * R_HV

lv_rated_current = 17.3

pf = [1.0, 0.8, 0.6, 0.4, 0.2]

df_dict = {
  'Power factor': pf,
}
df2 = pd.DataFrame(df_dict)

df2['RcosΦ'] = R_LV * df2['Power factor']
df2['XsinΦ'] = X_LV * ( (1 - (df2['Power factor'] ** 2)) ) ** 0.5
df2['RcosΦ + XsinΦ'] = df2['RcosΦ'] + df2['XsinΦ']
df2['RcosΦ - XsinΦ'] = df2['RcosΦ'] - df2['XsinΦ']
df2['Regulation (LAG)'] = lv_rated_current / lv_rated_voltage * 100 * df2['RcosΦ + XsinΦ']
df2['Regulation (LEAD)'] = lv_rated_current / lv_rated_voltage * 100 * df2['RcosΦ - XsinΦ']

display(df2.T)

,0,1,2,3,4
Power factor,1.000000,0.800000,0.600000,0.400000,0.200000
RcosΦ,0.211752,0.169402,0.127051,0.084701,0.042350
XsinΦ,0.000000,0.098324,0.131099,0.150193,0.160563
RcosΦ + XsinΦ,0.211752,0.267726,0.258150,0.234894,0.202913
RcosΦ - XsinΦ,0.211752,0.071077,-0.004048,-0.065492,-0.118212
Regulation (LAG),3.185488,4.027529,3.883477,3.533616,3.052520
Regulation (LEAD),3.185488,1.069253,-0.060891,-0.985225,-1.778324


# Show the values for the parameters of the equivalent circuit of the transformer

In [7]:
print('X_LV: ', X_LV, 'Ω')
print('R_LV: ', R_LV, 'Ω')
print('X_HV: ', X_HV, 'Ω')
print('R_HV: ', R_HV, 'Ω')

cos_phi_knot = W_o / (V_o * I_o)
I_w = I_o * cos_phi_knot
sin_phi_knot = (1 - (cos_phi_knot ** 2)) ** 0.5
I_m = I_o * (sin_phi_knot)

R_o = V_o / I_w
X_o = V_o / I_m

print('R_o (LV):', R_o)
print('X_o (LV):', X_o)

R_o = R_o / (transformation_ratio ** 2)
X_o = X_o / (transformation_ratio ** 2)

print('R_o (HV):', R_o)
print('X_o (HV):', X_o)

X_LV:  0.16387365743663498 Ω
R_LV:  0.21175211529686608 Ω
X_HV:  0.6554946297465399 Ω
R_HV:  0.8470084611874643 Ω
R_o (LV): 668.4765702891325
X_o (LV): 118.80257397663837
R_o (HV): 2673.90628115653
X_o (HV): 475.2102959065535


# Insert the observations for the Load Test, perform calculations and display the same

I had entered the data for No Load as S.No. 1 then 1/4th Load as S.No. 4, and since copying the values in the correct order from the incorrect order of my lab manual would be a pain, I just copied the values as it is and later passed an index list while making the dataframe instead.

In [8]:
V_1 = [116, 110.7, 110.0, 108.6, 115.1]
I_1 = [0.824, 9.41, 12.92, 16.01, 4.14]
W_1 = [0.023, 1.041, 1.425, 1.715, 0.472]
V_2 = [228.2, 212.4, 209.4, 202.6, 224.0]
I_2 = [0.0, 4.721, 6.523, 8.092, 1.999]
W_2 = [0.0, 1.003, 1.364, 1.634, 0.447]

n = len(V_1)
S_No = list(range(1, n+1, 1))

df_dict = {
  'S.No.': S_No,
  'Load': ['No Load', '1/2 Full Load', '3/4th Full Load', 'Full load', '1/4th Full Load Load'],
  'V₁ (V)': V_1,
  'I₁ (A)': I_1,
  'W₁ (kW)': W_1,
  'V₂ (V)': V_2,
  'I₂ (A)': I_2,
  'W₂ (kW)': W_2,
}

index_values = [0, 4, 1, 2, 3]

df = pd.DataFrame(df_dict)
df = df.reindex(index_values)
df.set_index('S.No.', inplace=True)

V_o2 = df['V₂ (V)'].iloc[0]# Secondary terminal voltage at no load

df['% efficiency'] = df['W₂ (kW)'] / df['W₁ (kW)'] * 100
df['% regulation'] = (V_o2 - df['V₂ (V)']) / V_o2 * 100

display(df)

,Load,V₁ (V),I₁ (A),W₁ (kW),V₂ (V),I₂ (A),W₂ (kW),% efficiency,% regulation
S.No.,,,,,,,,,
1,No Load,116.0,0.824,0.023,228.2,0.000,0.000,0.000000,0.000000
5,1/4th Full Load Load,115.1,4.140,0.472,224.0,1.999,0.447,94.703390,1.840491
2,1/2 Full Load,110.7,9.410,1.041,212.4,4.721,1.003,96.349664,6.923751
3,3/4th Full Load,110.0,12.920,1.425,209.4,6.523,1.364,95.719298,8.238387
4,Full load,108.6,16.010,1.715,202.6,8.092,1.634,95.276968,11.218230


# Show required plots

In [9]:
# length of the inteval is not an integer so can't use the range function to make a list,
# so we need to use linspace function of numpy for the x-axis (kW Power) tickvals
import numpy as np
import math

# utility function for the plotting function below.
# il stands for interval length, ul stands for upper limit, ll stands for lower limit
def get_range_and_tickvals(series, fractional_tickvals=False, num_intervals=10):
  # Formula used: il = ( int(series.max()) + il ) / num_intervals
  tickvals_list = []
  ul = None
  if fractional_tickvals==True:
    il = round(int(series.max()) / (num_intervals - 1), 2)
    ul = math.ceil(series.max()) + il
    ll = math.floor(series.min()) - il
    if ll >= 0:
      ll = 0
    tickvals_list = np.linspace(ll, ul, num=int((ul - ll) / il) + 1)
  else:
    il = math.ceil( int(series.max()) / (num_intervals - 1) )
    ul = math.ceil(series.max()) + il
    ll = math.floor(series.min()) - il
    if ll >= 0:
      ll = 0
    tickvals_list = list(range(ll, ul, il))
  return [ll, ul], tickvals_list

def show_plot(y_series, x_series, fractional_tickvals=False, reverse_x_axis=False):
  x_tickvals = []
  y_tickvals = []
  x_range = None
  y_range = None
  if fractional_tickvals==False:
    x_range, x_tickvals = get_range_and_tickvals(x_series)
    y_range, y_tickvals = get_range_and_tickvals(y_series)
  elif fractional_tickvals=='all':
    x_range, x_tickvals = get_range_and_tickvals(x_series, fractional_tickvals=True)
    y_range, y_tickvals = get_range_and_tickvals(y_series, fractional_tickvals=True)
  elif fractional_tickvals=='x-axis':
    x_range, x_tickvals = get_range_and_tickvals(x_series, fractional_tickvals=True)
    y_range, y_tickvals = get_range_and_tickvals(y_series)
  elif fractional_tickvals=='y-axis':
    x_range, x_tickvals = get_range_and_tickvals(x_series)
    y_range, y_tickvals = get_range_and_tickvals(y_series, fractional_tickvals=True)

  if reverse_x_axis==True:
    # convert x_tickvals to a list if its not already (i.e. its a numpy array)
    x_tickvals = x_tickvals if isinstance(x_tickvals, list) else list(x_tickvals)
    x_tickvals = x_tickvals[::-1]
    x_range = x_range[::-1]

  x_label = x_series.name
  y_label = y_series.name
  scatter = go.Scatter(x=x_series, y=y_series, mode='markers', name=f"{y_label} vs {x_label}")

  layout = go.Layout(
      title={'text': f"{y_label} vs {x_label}", 'x': 0.5, 'y': 0.95},
      xaxis=dict( title=f"{x_label}", titlefont_size=14, tickfont_size=12, range=x_range, tickvals=x_tickvals ),
      yaxis=dict( title=f"{y_label}", titlefont_size=14, tickfont_size=12, range=y_range, tickvals=y_tickvals ),
      autosize=False,
      width=600, height=400, margin=dict(l=100, r=50, b=100, t=100, pad=5),
      paper_bgcolor='LightSteelBlue',
      xaxis_showgrid=True,
      yaxis_showgrid=True,
  )

  fig = go.Figure(data=[scatter], layout=layout)

  fig.show(renderer="colab")

## Plot % Efficiency vs Output

In [10]:
show_plot(df1['Efficiency, % η'], df1['Output Power, P_o (kW)'], fractional_tickvals='x-axis')

## Plot % Regulation vs Power Factor

In [11]:
show_plot(df2['Regulation (LAG)'], df2['Power factor'], fractional_tickvals='all', reverse_x_axis=True)

In [12]:
show_plot(df2['Regulation (LEAD)'], df2['Power factor'], fractional_tickvals='all')